In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

# Для работы с матрицами
from scipy.sparse import csr_matrix

# Матричная факторизация
from implicit import als

# Модель второго уровня
from lightgbm import LGBMClassifier

import os, sys
module_path = os.path.abspath(os.path.join(os.pardir))
if module_path not in sys.path:
    sys.path.append(module_path)

import warnings
warnings.filterwarnings("ignore")
    
# Написанные нами функции
from metrics import precision_at_k, recall_at_k
from utils import prefilter_items
from recommenders import MainRecommender

## Read data

In [2]:
data = pd.read_csv('retail_train.csv')
item_features = pd.read_csv('product.csv')
user_features = pd.read_csv('hh_demographic.csv')

data_test = pd.read_csv('retail_test1.csv')

# Process features dataset

In [3]:
ITEM_COL = 'item_id'
USER_COL = 'user_id'

### Сolumn processing

In [4]:
item_features.columns = [col.lower() for col in item_features.columns]
user_features.columns = [col.lower() for col in user_features.columns]

item_features.rename(columns={'product_id': ITEM_COL}, inplace=True)
user_features.rename(columns={'household_key': USER_COL }, inplace=True)

# Split dataset for train, eval, test

In [5]:
# Важна схема обучения и валидации!
# -- давние покупки -- | -- 6 недель -- | -- 3 недель -- 
# подобрать размер 2-ого датасета (6 недель) --> learning curve (зависимость метрики recall@k от размера датасета)
VAL_MATCHER_WEEKS = 6
VAL_RANKER_WEEKS = 3

In [6]:
# берем данные для тренировки matching модели
data_train_matcher = data[(data['week_no'] >= data['week_no'].max() - (VAL_MATCHER_WEEKS + VAL_RANKER_WEEKS)) &
                      (data['week_no'] < data['week_no'].max() - (VAL_RANKER_WEEKS))]

# берем данные для валидации matching модели
data_val_matcher = data[data['week_no'] >= data['week_no'].max() - VAL_RANKER_WEEKS]

# берем данные для тренировки ranking модели
data_train_ranker = data[data['week_no'] >= data['week_no'].max() - VAL_RANKER_WEEKS]

# берем данные для теста ranking, matching модели
# data_val_ranker = data[data['week_no'] >= data['week_no'].max() - VAL_RANKER_WEEKS]
data_val_ranker = pd.read_csv('retail_test1.csv')

# сделаем объединенный сет данных для первого уровня (матчинга)
df_join_train_matcher = pd.concat([data_train_matcher, data_val_matcher])

In [7]:
def print_stats_data(df_data, name_df):
    print(name_df)
    print(f"Shape: {df_data.shape} Users: {df_data[USER_COL].nunique()} Items: {df_data[ITEM_COL].nunique()}  Min week: {df_data['week_no'].min()}   Max week: {df_data['week_no'].max()}" )

In [8]:
def print_stats_data_less(df_data, name_df):
    print(name_df)
    print(f"Shape: {df_data.shape} Users: {df_data[USER_COL].nunique()} Items: {df_data[ITEM_COL].nunique()}" )

In [9]:
print_stats_data(data_train_matcher,'train_matcher')
print_stats_data(data_val_matcher,'val_matcher')
print_stats_data(data_train_ranker,'train_ranker')
print_stats_data(data_val_ranker,'val_ranker')

print_stats_data(df_join_train_matcher,'join_train_matcher')

train_matcher
Shape: (169711, 12) Users: 2154 Items: 27649  Min week: 86   Max week: 91
val_matcher
Shape: (118314, 12) Users: 2042 Items: 24329  Min week: 92   Max week: 95
train_ranker
Shape: (118314, 12) Users: 2042 Items: 24329  Min week: 92   Max week: 95
val_ranker
Shape: (88734, 12) Users: 1885 Items: 20497  Min week: 96   Max week: 98
join_train_matcher
Shape: (288025, 12) Users: 2280 Items: 34332  Min week: 86   Max week: 95


In [10]:
data_train_matcher.head(2)

,user_id,basket_id,day,item_id,quantity,sales_value,store_id,retail_disc,trans_time,week_no,coupon_disc,coupon_match_disc
2104867,2070,40618492260,594,1019940,1,1.00,311,-0.29,40,86,0.0,0.0
2107468,2021,40618753059,594,840361,1,0.99,443,0.00,101,86,0.0,0.0


# Prefilter items

In [11]:
popularity_ = data_train_matcher.groupby('item_id')['quantity'].sum().reset_index()
popularity_.rename(columns={'quantity': 'n_sold'}, inplace=True)

popularity_.head(10)

,item_id,n_sold
0,28116,1
1,28117,1
2,28143,1
3,28186,1
4,28304,1
5,28966,1
6,29037,1
7,32111,1
8,32369,2
9,32785,3


#### Больше всего в колличественном выражении продется бензин, мы его в рассылках не рекомендуем, потому что это, скорее всего, заправка

In [12]:
top_n_sold = popularity_.loc[popularity_["n_sold"] >= 10000]
top_n_sold = pd.merge(top_n_sold, item_features, left_on='item_id', right_on='item_id')
top_n_sold

,item_id,n_sold,manufacturer,department,brand,commodity_desc,sub_commodity_desc,curr_size_of_product
0,397896,151803,69,KIOSK-GAS,Private,COUPON/MISC ITEMS,GASOLINE-REG UNLEADED,
1,480014,67338,69,KIOSK-GAS,Private,COUPON/MISC ITEMS,GASOLINE-REG UNLEADED,
2,707683,20226,69,MISC SALES TRAN,Private,COUPON/MISC ITEMS,GASOLINE-REG UNLEADED,
3,1404121,93280,69,KIOSK-GAS,Private,COUPON/MISC ITEMS,GASOLINE-REG UNLEADED,
4,1426702,33457,69,MISC SALES TRAN,Private,COUPON/MISC ITEMS,GASOLINE-REG UNLEADED,
5,5703832,40861,69,KIOSK-GAS,Private,COUPON/MISC ITEMS,GASOLINE-REG UNLEADED,
6,6410462,13851,69,KIOSK-GAS,Private,COUPON/MISC ITEMS,GASOLINE-REG UNLEADED,
7,6410464,11814,69,MISC SALES TRAN,Private,COUPON/MISC ITEMS,GASOLINE-REG UNLEADED,
8,6533889,1459599,69,MISC SALES TRAN,Private,COUPON/MISC ITEMS,GASOLINE-REG UNLEADED,
9,6534166,1028651,69,MISC SALES TRAN,Private,COUPON/MISC ITEMS,GASOLINE-REG UNLEADED,


In [13]:
top_n_sold = popularity_.loc[(popularity_["n_sold"] >= 500) & (popularity_["n_sold"] < 10000)]
top_n_sold = pd.merge(top_n_sold, item_features, left_on='item_id', right_on='item_id')
top_n_sold

,item_id,n_sold,manufacturer,department,brand,commodity_desc,sub_commodity_desc,curr_size_of_product
0,820165,553,2,PRODUCE,National,CITRUS,ORANGES NAVELS ALL,
1,883404,636,69,GROCERY,Private,BAKED BREAD/BUNS/ROLLS,MAINSTREAM WHITE BREAD,20 OZ
2,914190,860,317,GROCERY,National,CHEESE,CREAM CHEESE,L 8OZ
3,981760,671,69,GROCERY,Private,EGGS,EGGS - X-LARGE,1 DZ
4,995242,1185,69,GROCERY,Private,FLUID MILK PRODUCTS,FLUID MILK WHITE ONLY,
5,995785,548,2,PRODUCE,National,PEPPERS-ALL,PEPPERS GREEN BELL,48-54 CT
6,1013321,507,69,GROCERY,Private,VEGETABLES - SHELF STABLE,BEANS GREEN: FS/WHL/CUT,14.5 OZ
7,1029743,1271,69,GROCERY,Private,FLUID MILK PRODUCTS,FLUID MILK WHITE ONLY,1 GA
8,1082185,1879,2,PRODUCE,National,TROPICAL FRUIT,BANANAS,40 LB
9,1106523,850,69,GROCERY,Private,FLUID MILK PRODUCTS,FLUID MILK WHITE ONLY,1 GA


In [14]:
n_items_before = data_train_matcher['item_id'].nunique()
# Сокращаем выбор для модели первого уровня до 1500 наиболее популярных товаров т.к. большая часть ассортимента имеет единичные покупки.
data_train_matcher = prefilter_items(data_train_matcher, item_features=item_features, take_n_popular=1500)

n_items_after = data_train_matcher['item_id'].nunique()
print('Decreased # items from {} to {}'.format(n_items_before, n_items_after))

Decreased # items from 27649 to 1501


# Make cold-start to warm-start

In [15]:
# ищем общих пользователей
common_users = data_train_matcher.user_id.values

data_val_matcher = data_val_matcher[data_val_matcher.user_id.isin(common_users)]
data_train_ranker = data_train_ranker[data_train_ranker.user_id.isin(common_users)]
data_val_ranker = data_val_ranker[data_val_ranker.user_id.isin(common_users)]

print_stats_data(data_train_matcher,'train_matcher')
print_stats_data(data_val_matcher,'val_matcher')
print_stats_data(data_train_ranker,'train_ranker')
print_stats_data(data_val_ranker,'val_ranker')

train_matcher
Shape: (70033, 13) Users: 2100 Items: 1501  Min week: 86   Max week: 91
val_matcher
Shape: (115433, 12) Users: 1878 Items: 23961  Min week: 92   Max week: 95
train_ranker
Shape: (115433, 12) Users: 1878 Items: 23961  Min week: 92   Max week: 95
val_ranker
Shape: (85940, 12) Users: 1748 Items: 20210  Min week: 96   Max week: 98


# Init/train recommender

In [16]:
recommender = MainRecommender(data_train_matcher)

  0%|          | 0/15 [00:00<?, ?it/s]

  0%|          | 0/15 [00:00<?, ?it/s]

  0%|          | 0/15 [00:00<?, ?it/s]

  0%|          | 0/1501 [00:00<?, ?it/s]

# Eval recall of matching

In [17]:
ACTUAL_COL = 'actual'
result_eval_matcher = data_val_matcher.groupby(USER_COL)[ITEM_COL].unique().reset_index()
result_eval_matcher.columns=[USER_COL, ACTUAL_COL]
result_eval_matcher.head(2)

,user_id,actual
0,1,"[821867, 834484, 856942, 865456, 889248, 90795..."
1,6,"[920308, 926804, 946489, 1006718, 1017061, 107..."


In [18]:
def evalRecall(df_result, result_col_name, recommend_model, N):
    target_col_name = 'actual'
    df_result[result_col_name] = df_result['user_id'].apply(lambda x: recommend_model(x, N=N))
    return df_result.apply(lambda row: recall_at_k(row[result_col_name], row[target_col_name], k=N), axis=1).mean()

In [19]:
models = {'own_rec': recommender.get_own_recommendations,
          'similar_items': recommender.get_similar_items_recommendation,
          'als_rec': recommender.get_als_recommendations,
          'als_tfidf': recommender.get_als_tfidf_recommendations,
          'als_bm25': recommender.get_als_bm25_recommendations}

results = {'own_rec': [],
           'similar_items': [],
           'als_rec': [],
           'als_tfidf': [],
           'als_bm25': []}

### Обучим модель на N = 150

In [20]:
N_PREDICT = 150
result_eval_matcher = data_val_matcher.groupby(USER_COL)[ITEM_COL].unique().reset_index()
result_eval_matcher.columns=[USER_COL, ACTUAL_COL]

results = {'own_rec': [],
           'similar_items': [],
           'als_rec': [],
           'als_tfidf': [],
           'als_bm25': []}

result_eval_matcher.head(2)

,user_id,actual
0,1,"[821867, 834484, 856942, 865456, 889248, 90795..."
1,6,"[920308, 926804, 946489, 1006718, 1017061, 107..."


In [21]:
for model in models.keys():
    results[model].append(evalRecall(result_eval_matcher, model, models[model], N_PREDICT))

In [22]:
results

{'own_rec': [0.108651739513315],
 'similar_items': [0.08874889405648632],
 'als_rec': [0.07811716542650622],
 'als_tfidf': [0.08299980889727956],
 'als_bm25': [0.06699637778758959]}

### Recall@5 of matching

In [23]:
def calc_recall(df_data, top_k):
    for col_name in df_data.columns[2:]:
        yield col_name, df_data.apply(lambda row: recall_at_k(row[col_name], row[ACTUAL_COL], k=top_k), axis=1).mean()

        
def calc_precision(df_data, top_k):
    for col_name in df_data.columns[2:]:
        yield col_name, df_data.apply(lambda row: precision_at_k(row[col_name], row[ACTUAL_COL], k=top_k), axis=1).mean()

In [24]:
TOPK_RECALL = 50
TOPK_PRECISION = 5

In [25]:
sorted(calc_recall(result_eval_matcher, TOPK_RECALL), key=lambda x: x[1],reverse=True)

[('own_rec', 0.07201909364246917),
 ('als_tfidf', 0.051657214892853846),
 ('als_rec', 0.04850677096447502),
 ('similar_items', 0.04535916852731075),
 ('als_bm25', 0.03845214603731507)]

#### own_reccomended показывает лучший precision и recal, т.е. покупатель систематично покупает одни и те же товары. Далее будем рекоммендовать товары из own_reccomended и затем ранжировать

In [26]:
sorted(calc_precision(result_eval_matcher, TOPK_PRECISION), key=lambda x: x[1],reverse=True)

[('own_rec', 0.19499467518636848),
 ('als_tfidf', 0.16528221512247074),
 ('als_rec', 0.13120340788072418),
 ('als_bm25', 0.11970181043663472),
 ('similar_items', 0.04930777422790202)]

# Ranking part

### Обучаем модель 2-ого уровня на выбранных кандидатах

- Обучаем на data_train_ranking
- Обучаем *только* на выбранных кандидатах
- Лучший recall как увидели выше у get_own_recommendations, допустим сгенерировали 150 кандидатов (N_PREDICT = 150)

## Подготовка данных для трейна

In [27]:
# взяли пользователей из трейна для ранжирования
df_match_candidates = pd.DataFrame(data_train_ranker[USER_COL].unique())
df_match_candidates.columns = [USER_COL]

# собираем кандитатов с первого этапа (matcher), N_PREDICT = 150
df_match_candidates['candidates'] = df_match_candidates[USER_COL].apply(lambda x: recommender.get_own_recommendations(x, N=N_PREDICT))
df_match_candidates.head(2)

,user_id,candidates
0,338,"[5981072, 819840, 939770, 1068719, 960744, 102..."
1,2120,"[1029743, 1106523, 916122, 899624, 1044078, 10..."


In [28]:
# Делаем unpivot кандидатов для каждого юзера
df_items = df_match_candidates.apply(lambda x: pd.Series(x['candidates']), axis=1).stack().reset_index(level=1, drop=True)
df_items.name = 'item_id'

df_match_candidates = df_match_candidates.drop('candidates', axis=1).join(df_items)
df_match_candidates.head(4)

,user_id,item_id
0,338,5981072
0,338,819840
0,338,939770
0,338,1068719


### Check warm start

In [29]:
print_stats_data_less(df_match_candidates, 'match_candidates')

match_candidates
Shape: (281700, 2) Users: 1878 Items: 1479


### Создаем трейн сет для ранжирования с учетом кандидатов с этапа 1 

In [30]:
df_ranker_train = data_train_ranker[[USER_COL, ITEM_COL]].copy()
df_ranker_train['target'] = 1  # тут только покупки 

# Не хватает нулей в датасете, поэтому добавляем наших кандитатов в качество нулей
df_ranker_train = df_match_candidates.merge(df_ranker_train, on=[USER_COL, ITEM_COL], how='left')

# чистим дубликаты
df_ranker_train = df_ranker_train.drop_duplicates(subset=[USER_COL, ITEM_COL])
df_ranker_train['target'].fillna(0, inplace= True)

In [31]:
df_ranker_train.target.value_counts()

0.0    262573
1.0     10944
Name: target, dtype: int64

In [32]:
df_ranker_train.head(2)

,user_id,item_id,target
0,338,5981072,0.0
1,338,819840,0.0


(!) На каждого юзера 150 item_id-кандидатов

In [33]:
df_ranker_train['target'].mean()

0.04001213818519507

## Подготавливаем фичи для обучения модели

In [34]:
item_features.head(2)

,item_id,manufacturer,department,brand,commodity_desc,sub_commodity_desc,curr_size_of_product
0,25671,2,GROCERY,National,FRZN ICE,ICE - CRUSHED/CUBED,22 LB
1,26081,2,MISC. TRANS.,National,NO COMMODITY DESCRIPTION,NO SUBCOMMODITY DESCRIPTION,


In [35]:
user_features.head(2)

,age_desc,marital_status_code,income_desc,homeowner_desc,hh_comp_desc,household_size_desc,kid_category_desc,user_id
0,65+,A,35-49K,Homeowner,2 Adults No Kids,2,None/Unknown,1
1,45-54,A,50-74K,Homeowner,2 Adults No Kids,2,None/Unknown,7


#### Фичи user_id

In [36]:
# Средний чек
user_atv = df_join_train_matcher.groupby(['user_id', 'basket_id'])[['sales_value']].mean().reset_index().groupby('user_id')['sales_value'].mean()
user_atv.name = 'user_atv'
user_atv = np.round(user_atv, decimals=3)
user_atv.head()

user_id
1    3.035
2    3.086
3    2.069
4    4.588
6    3.315
Name: user_atv, dtype: float64

In [37]:
# Частотность покупок раз/месяц
df_join_train_matcher['month'] = df_join_train_matcher['day'].apply(lambda x: (x+30)//30)

avg_month_count = df_join_train_matcher.groupby(['user_id']).agg({ 'month': ['min', 'max'] }).reset_index()
avg_month_count['months_count'] = avg_month_count['month','max'] - avg_month_count['month','min']
avg_month_count.drop([('month','min'), ('month','max')], axis=1, inplace=True)
avg_month_count.set_index('user_id',inplace = True)

user_freq = df_join_train_matcher.groupby(['user_id'])['basket_id'].count() / avg_month_count['months_count']
user_freq.name = 'monthly_user_frequency'
user_freq = np.round(user_freq, decimals=3)
user_freq.head()

user_id
1    71.667
2       inf
3       inf
4       inf
6    55.333
Name: monthly_user_frequency, dtype: float64

In [38]:
# средняя стоимость покупок в месяц
user_sales_val = df_join_train_matcher.groupby(['user_id'])['sales_value'].sum().rename('monthly_user_salesval') / avg_month_count['months_count']
user_sales_val.name = 'monthly_user_salesval'
user_sales_val = np.round(user_sales_val, decimals=3)
user_sales_val.head()

user_id
1    180.633
2        inf
3        inf
4        inf
6    173.897
Name: monthly_user_salesval, dtype: float64

#### Фичи item_id

In [39]:
# Среднее кол-во покупок item_id в неделю
item_per_week = df_join_train_matcher.groupby(['item_id'])['quantity'].sum().rename('item_per_week') / df_join_train_matcher['week_no'].max() 
item_per_week = np.round(item_per_week, decimals=3)
item_per_week.head()

item_id
28116    0.011
28117    0.011
28143    0.011
28186    0.011
28304    0.011
Name: item_per_week, dtype: float64

In [40]:
# Средняя частота item в корзине
item_freq_per_basket = df_join_train_matcher.groupby(['item_id']).agg('user_id').count().rename('item_freq_per_basket')/df_join_train_matcher.basket_id.nunique()
item_freq_per_basket = np.round(item_freq_per_basket, decimals=4)
item_freq_per_basket.tail()

item_id
17991689    0.0000
17991691    0.0000
18000012    0.0001
18024155    0.0000
18024556    0.0001
Name: item_freq_per_basket, dtype: float64

In [41]:
# Среднее кол-во item в корзине
item_quantity_per_basket = df_join_train_matcher.groupby(['item_id']).agg('quantity').sum().rename('item_quantity_per_basket')/df_join_train_matcher.basket_id.nunique()
item_quantity_per_basket = np.round(item_quantity_per_basket, decimals=4)
item_quantity_per_basket.tail()

item_id
17991689    0.0000
17991691    0.0000
18000012    0.0001
18024155    0.0000
18024556    0.0001
Name: item_quantity_per_basket, dtype: float64

In [42]:
# Цена (Можно посчитать из retil_train.csv) - цена есть в таблице взаимодействий
item_price = df_join_train_matcher.groupby(['item_id']).agg({'quantity': 'sum', 'sales_value': 'sum'})
item_price = item_price['sales_value'] / item_price['quantity']
item_price.name = 'item_price'
item_price = np.round(item_price, decimals=2)
item_price.head()

item_id
28116    0.33
28117    0.34
28143    0.33
28186    0.79
28304    0.53
Name: item_price, dtype: float64

In [43]:
# Среднее кол-во покупок 1 товара в категории в неделю
data_categ = pd.merge(df_join_train_matcher, item_features, how='inner', on='item_id')

categ_freq_ = data_categ.groupby(['department'])['quantity'].sum()
categ_item_freq_ = data_categ.groupby(['department','item_id'])['quantity'].sum().reset_index()

categ_freq = pd.merge(categ_item_freq_, categ_freq_, on='department')
categ_freq['category_freq'] = categ_freq['quantity_y'] / df_join_train_matcher['week_no'].max() / categ_freq['quantity_x']
categ_freq.set_index('item_id', inplace=True)
category_freq = categ_freq['category_freq']
category_freq = np.round(category_freq, decimals=3)
category_freq.tail()

item_id
17214973    3.063
471772      0.421
1070538     0.011
1847410     0.842
15681643    0.011
Name: category_freq, dtype: float64

In [44]:
# Генерируем признак - is_gasoline
item_features['is_gasoline'] = np.where(item_features['sub_commodity_desc'] == 'GASOLINE-REG UNLEADED', 1, 0)

In [45]:
# (Кол-во покупок в неделю) / (Среднее кол-во покупок 1 товара в категории в неделю)
freq_ratio_1 = item_per_week / category_freq
freq_ratio_1.name = 'freq_ratio_1'
freq_ratio_1 = np.round(freq_ratio_1, decimals=3)
freq_ratio_1.tail()

item_id
17991689    0.000
17991691    0.000
18000012    0.031
18024155    0.000
18024556    0.000
Name: freq_ratio_1, dtype: float64

In [46]:
# Цена / Средняя цена товара в категории
data_categ['price'] = data_categ['sales_value']/data_categ['quantity']
avg_categ_price = data_categ.groupby(['commodity_desc'])['price'].mean()
data_categ_avg = pd.merge(data_categ, avg_categ_price, on='commodity_desc', suffixes=('_x', '_y'))
data_categ_avg = data_categ_avg.groupby(['item_id']).agg({'price_x': 'mean', 'price_y': 'mean'})

price_diff = data_categ_avg['price_x'] / data_categ_avg['price_y']
price_diff.name = 'price_diff'
price_diff = np.round(price_diff, decimals=3)
price_diff.head()

item_id
28116    0.372
28117    0.384
28143    0.372
28186    0.590
28304    0.420
Name: price_diff, dtype: float64

#### Фичи пары user_id - item_id

In [47]:
# (Кол-во покупок юзером конкретной категории в неделю) / (Среднее кол-во покупок всеми юзерами конкретной категории в неделю)

categ_user_freq_ = data_categ.groupby(['user_id', 'department'])['quantity'].sum().reset_index()
categ_user_freq_['category_freq'] = categ_user_freq_['quantity'] / data_train_matcher['week_no'].max()

categ_user_freq = pd.merge(categ_user_freq_, categ_freq_, on='department')
categ_user_freq['avg_weekly_freq'] = categ_user_freq['quantity_y'] / data_train_matcher['week_no'].max()
categ_user_freq['freq_ratio_2'] = categ_user_freq['category_freq'] / categ_user_freq['avg_weekly_freq']

user_dep_ratio_2 = categ_user_freq.filter(['user_id', 'department', 'freq_ratio_2'], axis=1)
user_dep_ratio_2 = np.round(user_dep_ratio_2, decimals=5)
user_dep_ratio_2.tail()

,user_id,department,freq_ratio_2
16632,886,AUTOMOTIVE,0.33333
16633,1379,AUTOMOTIVE,0.33333
16634,820,VIDEO,1.00000
16635,2168,PHOTO,0.50000
16636,2322,PHOTO,0.50000


In [48]:
# (Кол-во покупок юзером конкретной категории в неделю) - (Среднее кол-во покупок всеми юзерами конкретной категории в неделю)
categ_user_freq['freq_ratio_3'] = categ_user_freq['category_freq'] - categ_user_freq['avg_weekly_freq']
user_dep_ratio_3 = categ_user_freq.filter(['user_id', 'department', 'freq_ratio_3'], axis=1)
user_dep_ratio_3 = np.round(user_dep_ratio_3, decimals=2)
user_dep_ratio_3.tail()

,user_id,department,freq_ratio_3
16632,886,AUTOMOTIVE,-0.02
16633,1379,AUTOMOTIVE,-0.02
16634,820,VIDEO,0.00
16635,2168,PHOTO,-0.01
16636,2322,PHOTO,-0.01


In [49]:
# Также добавляем блок сгенерированных быстрых фичей
df_ranker_train = df_ranker_train.merge(df_join_train_matcher.groupby(by=ITEM_COL).agg('sales_value').sum().rename('total_item_sales_value'), how='left',on=ITEM_COL)

df_ranker_train = df_ranker_train.merge(df_join_train_matcher.groupby(by=ITEM_COL).agg('quantity').sum().rename('total_quantity_value'), how='left',on=ITEM_COL)

df_ranker_train = df_ranker_train.merge(df_join_train_matcher.groupby(by=ITEM_COL).agg('sales_value').median().rename('avg_item_sales_value'), how='left',on=ITEM_COL)

df_ranker_train = df_ranker_train.merge(df_join_train_matcher.groupby(by=ITEM_COL).agg('quantity').median().rename('avg_quantity_value'), how='left',on=ITEM_COL)

df_ranker_train = df_ranker_train.merge(df_join_train_matcher.groupby(by=USER_COL).agg('sales_value').median().rename('avg_user_sales_value'), how='left',on=USER_COL)

df_ranker_train = df_ranker_train.merge(df_join_train_matcher.groupby(by=USER_COL).agg('quantity').median().rename('avg_quantity_value'), how='left',on=USER_COL)

df_ranker_train = df_ranker_train.merge(df_join_train_matcher.groupby(by=ITEM_COL).agg('quantity').median().rename('avg_item_quantity_per_week')/df_join_train_matcher.week_no.nunique(), how='left',on=ITEM_COL)

df_ranker_train = df_ranker_train.merge(df_join_train_matcher.groupby(by=USER_COL).agg('quantity').median().rename('avg_user_quantity_per_week')/df_join_train_matcher.week_no.nunique(), how='left',on=USER_COL)

df_ranker_train = df_ranker_train.merge(df_join_train_matcher.groupby(by=USER_COL).agg('quantity').sum().rename('user_quantity_per_baskter')/df_join_train_matcher.basket_id.nunique(), how='left',on=USER_COL)

df_ranker_train = df_ranker_train.merge(df_join_train_matcher.groupby(by=USER_COL).agg(USER_COL).count().rename('user_freq_per_basket')/df_join_train_matcher.basket_id.nunique(), how='left',on=USER_COL)

df_ranker_train = df_ranker_train.merge(df_join_train_matcher.groupby(by=USER_COL).agg('sales_value').sum().rename('total_user_sales_value'), how='left',on=USER_COL)

df_ranker_train = df_ranker_train.merge(df_join_train_matcher.groupby(by=ITEM_COL).agg(USER_COL).count().rename('item_freq'), how='left',on=ITEM_COL)

df_ranker_train = df_ranker_train.merge(df_join_train_matcher.groupby(by=USER_COL).agg(USER_COL).count().rename('user_freq'), how='left',on=USER_COL)

df_ranker_train = df_ranker_train.merge(df_join_train_matcher.groupby(by=ITEM_COL).agg('quantity').sum().rename('item_quantity_per_week')/df_join_train_matcher.week_no.nunique(), how='left',on=ITEM_COL)

df_ranker_train = df_ranker_train.merge(df_join_train_matcher.groupby(by=USER_COL).agg('quantity').sum().rename('user_quantity_per_week')/df_join_train_matcher.week_no.nunique(), how='left',on=USER_COL)

## Добавляем сгенерированные фичи в таблицы 

In [50]:
user_features = user_features.merge(user_atv, how='left', on='user_id', suffixes=(False, False))
user_features = user_features.merge(user_freq, how='left', on='user_id', suffixes=(False, False))
user_features = user_features.merge(user_sales_val, how='left', on='user_id', suffixes=(False, False))

In [51]:
user_features.head()

,age_desc,marital_status_code,income_desc,homeowner_desc,hh_comp_desc,household_size_desc,kid_category_desc,user_id,user_atv,monthly_user_frequency,monthly_user_salesval
0,65+,A,35-49K,Homeowner,2 Adults No Kids,2,None/Unknown,1,3.035,71.667,180.633
1,45-54,A,50-74K,Homeowner,2 Adults No Kids,2,None/Unknown,7,2.658,110.500,267.605
2,25-34,U,25-34K,Unknown,2 Adults Kids,3,1,8,2.689,98.333,242.660
3,25-34,U,75-99K,Homeowner,2 Adults Kids,4,2,13,18.274,104.667,623.673
4,45-54,B,50-74K,Homeowner,Single Female,1,None/Unknown,16,2.925,10.000,30.410


In [52]:
item_features = item_features.merge(item_per_week, how='left', on='item_id', suffixes=(False, False))
item_features = item_features.merge(category_freq, how='left', on='item_id', suffixes=(False, False))
item_features = item_features.merge(freq_ratio_1, how='left', on='item_id', suffixes=(False, False))
item_features = item_features.merge(price_diff, how='left', on='item_id', suffixes=(False, False))
item_features = item_features.merge(item_freq_per_basket, how='left', on='item_id', suffixes=(False, False))
item_features = item_features.merge(item_quantity_per_basket, how='left', on='item_id', suffixes=(False, False))

In [53]:
# Некоторых товаров не было в транзакциях тренировочного датасета
item_features[item_features['category_freq'].isna()].head()

,item_id,manufacturer,department,brand,commodity_desc,sub_commodity_desc,curr_size_of_product,is_gasoline,item_per_week,category_freq,freq_ratio_1,price_diff,item_freq_per_basket,item_quantity_per_basket
0,25671,2,GROCERY,National,FRZN ICE,ICE - CRUSHED/CUBED,22 LB,0,NaN,NaN,NaN,NaN,NaN,NaN
1,26081,2,MISC. TRANS.,National,NO COMMODITY DESCRIPTION,NO SUBCOMMODITY DESCRIPTION,,0,NaN,NaN,NaN,NaN,NaN,NaN
2,26093,69,PASTRY,Private,BREAD,BREAD:ITALIAN/FRENCH,,0,NaN,NaN,NaN,NaN,NaN,NaN
3,26190,69,GROCERY,Private,FRUIT - SHELF STABLE,APPLE SAUCE,50 OZ,0,NaN,NaN,NaN,NaN,NaN,NaN
4,26355,69,GROCERY,Private,COOKIES/CONES,SPECIALTY COOKIES,14 OZ,0,NaN,NaN,NaN,NaN,NaN,NaN


In [54]:
df_ranker_train = df_ranker_train.merge(item_features, on='item_id', how='left', suffixes=(False, False))
df_ranker_train = df_ranker_train.merge(user_features, on='user_id', how='left', suffixes=(False, False))

In [55]:
df_ranker_train = pd.merge(df_ranker_train, user_dep_ratio_2, on=['user_id', 'department'], how='left')
df_ranker_train = pd.merge(df_ranker_train, user_dep_ratio_3, on=['user_id', 'department'], how='left')

In [56]:
X_train = df_ranker_train.drop('target', axis=1)
y_train = df_ranker_train[['target']]

In [57]:
cat_feats = X_train.columns[2:].tolist()
X_train[cat_feats] = X_train[cat_feats].astype('category')

cat_feats

['total_item_sales_value',
 'total_quantity_value',
 'avg_item_sales_value',
 'avg_quantity_value_x',
 'avg_user_sales_value',
 'avg_quantity_value_y',
 'avg_item_quantity_per_week',
 'avg_user_quantity_per_week',
 'user_quantity_per_baskter',
 'user_freq_per_basket',
 'total_user_sales_value',
 'item_freq',
 'user_freq',
 'item_quantity_per_week',
 'user_quantity_per_week',
 'manufacturer',
 'department',
 'brand',
 'commodity_desc',
 'sub_commodity_desc',
 'curr_size_of_product',
 'is_gasoline',
 'item_per_week',
 'category_freq',
 'freq_ratio_1',
 'price_diff',
 'item_freq_per_basket',
 'item_quantity_per_basket',
 'age_desc',
 'marital_status_code',
 'income_desc',
 'homeowner_desc',
 'hh_comp_desc',
 'household_size_desc',
 'kid_category_desc',
 'user_atv',
 'monthly_user_frequency',
 'monthly_user_salesval',
 'freq_ratio_2',
 'freq_ratio_3']

## Обучение модели ранжирования

In [58]:
lgb = LGBMClassifier(objective='binary',
                     max_depth=100,
                     n_estimators=300,
                     learning_rate=0.1,
                     categorical_column=cat_feats)

lgb.fit(X_train, y_train)

train_preds = lgb.predict_proba(X_train)

In [59]:
df_ranker_predict = df_ranker_train.copy()

In [60]:
df_ranker_predict['proba_item_purchase'] = train_preds[:,1]

In [61]:
def popularity_recommendation(data, item_features, n):
    """Топ-n популярных товаров из уникальной категории"""
    
    popular = data.groupby('item_id')['proba_item_purchase'].sum().reset_index()
    popular = pd.merge(popular, item_features, left_on='item_id', right_on='item_id', suffixes=(False, False))
    popular = popular.nlargest(15,'proba_item_purchase')
    popular = popular.groupby('commodity_desc').max('proba_item_purchase')
    
    popular.sort_values('proba_item_purchase', ascending=False, inplace=True)
    recs = popular.head(n).item_id
    
    return recs.tolist()

In [62]:
popular_recs = popularity_recommendation(df_ranker_predict, item_features = item_features, n=5)
popular_recs

[1082185, 1106523, 5569230, 951590, 1004906]

### Мы обучили модель ранжирования на покупках из сета data_train_ranker и на кандитатах от own_recommendations

# Evaluation on test dataset

In [63]:
# Преобразуем тестовый датасет в нужный формат
result_eval_ranker = data_val_ranker.groupby(USER_COL)[ITEM_COL].unique().reset_index()
result_eval_ranker.columns=[USER_COL, ACTUAL_COL]
result_eval_ranker.head(2)

,user_id,actual
0,1,"[880007, 883616, 931136, 938004, 940947, 94726..."
1,2,"[820165, 820291, 826784, 826835, 829009, 85784..."


## Eval re-ranked matched result on test dataset  

In [64]:
def rerank(user_id, N):
    return df_ranker_predict[df_ranker_predict[USER_COL]==user_id].sort_values('proba_item_purchase', ascending=False).head(N).item_id.tolist()

In [65]:
# precision вырос в двухуровневой модели по сравнению с одноуровневым own_recommend 
result_eval_ranker['reranked_own_rec'] = result_eval_ranker[USER_COL].apply(lambda user_id: rerank(user_id, 5))
print(*sorted(calc_precision(result_eval_ranker, TOPK_PRECISION), key=lambda x: x[1], reverse=True), sep='\n')

('reranked_own_rec', 0.24485070079219992)


In [66]:
result_eval_ranker.head(3)

,user_id,actual,reranked_own_rec
0,1,"[880007, 883616, 931136, 938004, 940947, 94726...","[1082185, 856942, 8293439, 940947, 1132771]"
1,2,"[820165, 820291, 826784, 826835, 829009, 85784...",[]
2,6,"[956902, 960791, 1037863, 1119051, 1137688, 84...","[1082185, 845208, 1024306, 5569230, 1004906]"


# Оценка на тестовом датасете 

In [67]:
# тестовый датасет
df_test = pd.read_csv('retail_test1.csv')

In [68]:
result_test = df_test.groupby(USER_COL)[ITEM_COL].unique().reset_index()
result_test.columns=[USER_COL, ACTUAL_COL]
result_test.head(2)

,user_id,actual
0,1,"[880007, 883616, 931136, 938004, 940947, 94726..."
1,2,"[820165, 820291, 826784, 826835, 829009, 85784..."


In [69]:
result_test.shape

(1885, 2)

In [70]:
# Покупатели, которые были в тренировочных датасетах
warm_start = result_test[result_test.user_id.isin(common_users)]
warm_start.shape

(1748, 2)

In [71]:
# Пользователям, которых не было в тренировочных датасетах, далее предложим popular recommended товары
cold_start = result_test[~result_test.user_id.isin(common_users)]
cold_start.shape

(137, 2)

In [72]:
warm_start['own_rec'] = warm_start[USER_COL].apply(lambda x: recommender.get_own_recommendations(x, N=N_PREDICT))

In [73]:
# precision@5 в двухуровневой модели значительно выше, целевое значение достигнуто

warm_start['reranked_own_rec'] = warm_start[USER_COL].apply(lambda user_id: rerank(user_id, 5))
print(*sorted(calc_precision(warm_start, TOPK_PRECISION), key=lambda x: x[1], reverse=True), sep='\n')

('reranked_own_rec', 0.24485070079219992)
('own_rec', 0.15663615560640734)


In [74]:
# Пользователи, по которым не было истории для предикта модели второго уровня (ранжирования)
result_nan = warm_start.loc[warm_start["reranked_own_rec"].str.len() == 0]
user_list = result_nan['user_id'].unique()
len(user_list)

107

In [75]:
# Им можно рекоммендовать товары из popular_recs
warm_start.loc[warm_start["user_id"].isin(user_list), "reranked_own_rec"] =\
warm_start.loc[warm_start["user_id"].isin(user_list), "user_id"].apply(lambda x: popular_recs)
warm_start.head()

,user_id,actual,own_rec,reranked_own_rec
0,1,"[880007, 883616, 931136, 938004, 940947, 94726...","[8293439, 856942, 9655212, 922281, 940947, 109...","[1082185, 856942, 8293439, 940947, 1132771]"
1,2,"[820165, 820291, 826784, 826835, 829009, 85784...","[6633224, 839656, 1139142, 1011457, 1030981, 8...","[1082185, 1106523, 5569230, 951590, 1004906]"
3,6,"[956902, 960791, 1037863, 1119051, 1137688, 84...","[1024306, 12757544, 6548453, 983096, 850102, 8...","[1082185, 845208, 1024306, 5569230, 1004906]"
4,7,"[847270, 855557, 859987, 863407, 895454, 90663...","[991223, 5572738, 1023196, 9836752, 993638, 10...","[1082185, 5568378, 5592610, 1122358, 986912]"
5,8,"[846334, 850834, 857503, 862139, 865891, 87829...","[1042544, 1013389, 914318, 1121694, 866355, 94...","[1082185, 1029743, 854405, 872137, 839243]"


In [76]:
print(*sorted(calc_precision(warm_start, TOPK_PRECISION), key=lambda x: x[1], reverse=True), sep='\n')

('reranked_own_rec', 0.2344393592677346)
('own_rec', 0.15663615560640734)


In [77]:
# Или можно рекоммендовать товары из own_recommender
warm_start.loc[warm_start["user_id"].isin(user_list), "reranked_own_rec"] =\
warm_start.loc[warm_start["user_id"].isin(user_list), "user_id"].apply(lambda x: recommender.get_own_recommendations(x, N=5))
warm_start.head()

,user_id,actual,own_rec,reranked_own_rec
0,1,"[880007, 883616, 931136, 938004, 940947, 94726...","[8293439, 856942, 9655212, 922281, 940947, 109...","[1082185, 856942, 8293439, 940947, 1132771]"
1,2,"[820165, 820291, 826784, 826835, 829009, 85784...","[6633224, 839656, 1139142, 1011457, 1030981, 8...","[6633224, 839656, 1139142, 1011457, 1030981]"
3,6,"[956902, 960791, 1037863, 1119051, 1137688, 84...","[1024306, 12757544, 6548453, 983096, 850102, 8...","[1082185, 845208, 1024306, 5569230, 1004906]"
4,7,"[847270, 855557, 859987, 863407, 895454, 90663...","[991223, 5572738, 1023196, 9836752, 993638, 10...","[1082185, 5568378, 5592610, 1122358, 986912]"
5,8,"[846334, 850834, 857503, 862139, 865891, 87829...","[1042544, 1013389, 914318, 1121694, 866355, 94...","[1082185, 1029743, 854405, 872137, 839243]"


In [78]:
# C own recommender получаем чуть более высокий precision 
print(*sorted(calc_precision(warm_start, TOPK_PRECISION), key=lambda x: x[1], reverse=True), sep='\n')

('reranked_own_rec', 0.23489702517162472)
('own_rec', 0.15663615560640734)


In [79]:
# Остальным покупателям рекомендуем популярные товары 
cold_start['own_rec'] = cold_start.apply(lambda row: popular_recs, axis = 1)
cold_start['reranked_own_rec'] = cold_start.apply(lambda row: popular_recs, axis = 1)
cold_start

,user_id,actual,own_rec,reranked_own_rec
2,3,"[827683, 908531, 989069, 1071377, 1080155, 109...","[1082185, 1106523, 5569230, 951590, 1004906]","[1082185, 1106523, 5569230, 951590, 1004906]"
7,10,"[871162, 962118, 986912, 1043766, 1068504, 110...","[1082185, 1106523, 5569230, 951590, 1004906]","[1082185, 1106523, 5569230, 951590, 1004906]"
36,43,"[821787, 1029174, 1095068, 873178, 838319, 925...","[1082185, 1106523, 5569230, 951590, 1004906]","[1082185, 1106523, 5569230, 951590, 1004906]"
40,50,"[833245, 835058, 838602, 865992, 874736, 89984...","[1082185, 1106523, 5569230, 951590, 1004906]","[1082185, 1106523, 5569230, 951590, 1004906]"
56,70,[1039156],"[1082185, 1106523, 5569230, 951590, 1004906]","[1082185, 1106523, 5569230, 951590, 1004906]"
...,...,...,...,...
1791,2368,"[838210, 843346, 845992, 857540, 868745, 87822...","[1082185, 1106523, 5569230, 951590, 1004906]","[1082185, 1106523, 5569230, 951590, 1004906]"
1795,2373,"[838602, 1029743, 1109598]","[1082185, 1106523, 5569230, 951590, 1004906]","[1082185, 1106523, 5569230, 951590, 1004906]"
1814,2399,"[974156, 986912]","[1082185, 1106523, 5569230, 951590, 1004906]","[1082185, 1106523, 5569230, 951590, 1004906]"
1829,2425,"[836181, 929751, 994272, 995242, 999104, 10288...","[1082185, 1106523, 5569230, 951590, 1004906]","[1082185, 1106523, 5569230, 951590, 1004906]"


In [80]:
# Объединяем пользователей тестового датафрейма
result_df = warm_start.append(cold_start)
result_df 

,user_id,actual,own_rec,reranked_own_rec
0,1,"[880007, 883616, 931136, 938004, 940947, 94726...","[8293439, 856942, 9655212, 922281, 940947, 109...","[1082185, 856942, 8293439, 940947, 1132771]"
1,2,"[820165, 820291, 826784, 826835, 829009, 85784...","[6633224, 839656, 1139142, 1011457, 1030981, 8...","[6633224, 839656, 1139142, 1011457, 1030981]"
3,6,"[956902, 960791, 1037863, 1119051, 1137688, 84...","[1024306, 12757544, 6548453, 983096, 850102, 8...","[1082185, 845208, 1024306, 5569230, 1004906]"
4,7,"[847270, 855557, 859987, 863407, 895454, 90663...","[991223, 5572738, 1023196, 9836752, 993638, 10...","[1082185, 5568378, 5592610, 1122358, 986912]"
5,8,"[846334, 850834, 857503, 862139, 865891, 87829...","[1042544, 1013389, 914318, 1121694, 866355, 94...","[1082185, 1029743, 854405, 872137, 839243]"
...,...,...,...,...
1791,2368,"[838210, 843346, 845992, 857540, 868745, 87822...","[1082185, 1106523, 5569230, 951590, 1004906]","[1082185, 1106523, 5569230, 951590, 1004906]"
1795,2373,"[838602, 1029743, 1109598]","[1082185, 1106523, 5569230, 951590, 1004906]","[1082185, 1106523, 5569230, 951590, 1004906]"
1814,2399,"[974156, 986912]","[1082185, 1106523, 5569230, 951590, 1004906]","[1082185, 1106523, 5569230, 951590, 1004906]"
1829,2425,"[836181, 929751, 994272, 995242, 999104, 10288...","[1082185, 1106523, 5569230, 951590, 1004906]","[1082185, 1106523, 5569230, 951590, 1004906]"


In [81]:
# Получившийся precision@5 по всем пользователям
print(*sorted(calc_precision(result_df , TOPK_PRECISION), key=lambda x: x[1], reverse=True), sep='\n')

('reranked_own_rec', 0.2223872679045093)
('own_rec', 0.14981432360742708)
